In [1]:
# Import required libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

# Read in the insurance dataset
data = pd.read_csv("/content/drive/MyDrive/Car details.csv")

In [3]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [4]:
data.columns

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model


In [15]:
data = pd.read_csv("/content/drive/MyDrive/Car details.csv")

In [16]:
data = data.dropna()


In [17]:
X = data.drop(columns=['selling_price'])
y = data['selling_price']


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
preprocessor = make_column_transformer(
    (MinMaxScaler(), ['year', 'km_driven', 'seats']),
    (OneHotEncoder(handle_unknown='ignore'), ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque'])
)


In [20]:
preprocessor.fit(X_train)


ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['year', 'km_driven', 'seats']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['name', 'fuel', 'seller_type', 'transmission',
                                  'owner', 'mileage', 'engine', 'max_power',
                                  'torque'])])

In [21]:
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


In [24]:
X_train_transformed = preprocessor.transform(X_train).toarray()
X_test_transformed = preprocessor.transform(X_test).toarray()


In [25]:
X_train_transformed, X_val_transformed, y_train, y_val = train_test_split(X_train_transformed, y_train, test_size=0.1, random_state=42)


In [36]:
def create_model():
    input_layer = Input(shape=(X_train_transformed.shape[1],))
    dense1 = Dense(128, activation='relu')(input_layer)
    dense2 = Dense(64, activation='relu')(dense1)
    dense3 = Dense(32, activation='relu')(dense2)
    output_layer = Dense(1)(dense3)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

model = create_model()


In [37]:
model.fit(X_train_transformed, y_train, epochs=200, batch_size=32, validation_data=(X_val_transformed, y_val))


Epoch 1/200
178/178 [==============================] - 4s 14ms/step - loss: 646372.1250 - val_loss: 659293.4375
Epoch 2/200
178/178 [==============================] - 2s 8ms/step - loss: 566528.8750 - val_loss: 465922.1250
Epoch 3/200
178/178 [==============================] - 1s 8ms/step - loss: 372787.2188 - val_loss: 362867.5625
Epoch 4/200
178/178 [==============================] - 1s 8ms/step - loss: 323119.3750 - val_loss: 330754.9688
Epoch 5/200
178/178 [==============================] - 1s 7ms/step - loss: 292387.5000 - val_loss: 301368.2188
Epoch 6/200
178/178 [==============================] - 1s 8ms/step - loss: 269034.2812 - val_loss: 281306.4375
Epoch 7/200
178/178 [==============================] - 1s 8ms/step - loss: 252902.6250 - val_loss: 267031.4688
Epoch 8/200
178/178 [==============================] - 1s 7ms/step - loss: 239984.6406 - val_loss: 254457.9062
Epoch 9/200
178/178 [==============================] - 2s 10ms/step - loss: 228466.7344 - val_loss: 244300.1250

In [38]:
test_mae = model.evaluate(X_test_transformed, y_test)
print(f"Mean absolute error on test data: {test_mae}")


50/50 [==============================] - 0s 3ms/step - loss: 71373.1094
Mean absolute error on test data: 71373.109375


In [39]:
def predict_random_test_data(model, X_test, X_test_transformed, y_test):
    random_index = np.random.choice(X_test_transformed.shape[0])
    random_test_data = X_test_transformed[random_index]
    true_selling_price = y_test.iloc[random_index]

    prediction = model.predict(random_test_data[np.newaxis, :])[0][0]
    
    print("Test data parameters:")
    print(X_test.iloc[random_index])
    print(f"\nTrue selling price: {true_selling_price}")
    print(f"Predicted selling price: {prediction}")


In [47]:
predict_random_test_data(model, X_test, X_test_transformed, y_test)


1/1 [==============================] - 0s 33ms/step
Test data parameters:
name            Tata Indigo CS LX (TDI) BS-III
year                                      2015
km_driven                                80000
fuel                                    Diesel
seller_type                         Individual
transmission                            Manual
owner                             Second Owner
mileage                             19.09 kmpl
engine                                 1405 CC
max_power                            69.01 bhp
torque                          135Nm@ 2500rpm
seats                                      5.0
Name: 5654, dtype: object

True selling price: 250000
Predicted selling price: 249512.421875


In [60]:
def predict_random_test_data(model, X_test, X_test_transformed, y_test):
    random_index = np.random.choice(X_test_transformed.shape[0])
    random_test_data = X_test_transformed[random_index]
    true_selling_price = y_test.iloc[random_index]

    prediction = model.predict(random_test_data[np.newaxis, :])[0][0]
    
    print("Test data parameters:")
    display(X_test.iloc[[random_index]])
    
    print(f"\nTrue selling price: {true_selling_price}")
    print(f"Predicted selling price: {round(prediction, 2):,.2f}")


In [68]:
from IPython.display import display

predict_random_test_data(model, X_test, X_test_transformed, y_test)


1/1 [==============================] - 0s 24ms/step
Test data parameters:


,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
2664,Lexus ES 300h,2019,20000,Petrol,Dealer,Automatic,First Owner,22.37 kmpl,2487 CC,214.56 bhp,202Nm@ 3600-5200rpm,5.0



True selling price: 5150000
Predicted selling price: 5,170,161.00
